In [2]:
# Import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import json
import time
from pandas.io.json import json_normalize

# The plots will be style using the seaborm library.
import seaborn as sns

# Use seaborn style defaults and set the default figure size
sns.set(rc={'figure.figsize':(11, 4)})

# Import a new dependency to format the dates
import matplotlib.dates as mdates

In [3]:
#Get data from Alpha Vantage using API request

url = "https://www.alphavantage.co/query"
params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": "CL",
        "outputsize": "full",
        "apikey": "ac9ba10aa04a7d"}



response = requests.get(url, params=params)
data = response.json()

def df_from_response(data,
         columns=['Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume']):

    def convert_response(d):
        # convert the response into datetimerecords that can be
        # parsed by Pandas
        for dt, prec in d['Time Series (Daily)'].items():
            r = { 'date': dt}
            r.update(prec)
            yield r
               
    # pass your response 'page'
    df = pd.DataFrame(convert_response(data))
    # rename the columns    
    df = df.rename(columns={'1. open': 'Open',
                            '2. high': 'High',
                            '3. low': 'Low',
                            '4. close': 'Close',
                            '5. adjusted close': 'AdjClose',
                            '6. volume': 'Volume'})
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df.sort_index(inplace=True)
    # extract the default columns
    df = df[columns]
    return df

df = df_from_response(data)
df.reset_index(level=0, inplace=True)
df.head(2)

,date,Open,High,Low,Close,AdjClose,Volume
0,1998-01-02,74.5000,74.5000,72.6900,73.2500,11.9706,451500
1,1998-01-05,74.0000,74.8800,73.2500,74.8800,12.2369,770000


In [4]:
#Convert results into a dataframe and assign an index
df1 = pd.DataFrame(df)
df1.head()

,date,Open,High,Low,Close,AdjClose,Volume
0,1998-01-02,74.5000,74.5000,72.6900,73.2500,11.9706,451500
1,1998-01-05,74.0000,74.8800,73.2500,74.8800,12.2369,770000
2,1998-01-06,74.9400,74.9400,71.7500,71.9400,11.7565,861600
3,1998-01-07,70.2500,71.6300,69.8800,71.6300,11.7058,1125100
4,1998-01-08,71.6300,71.7500,69.3800,69.7500,11.3986,949700


In [5]:
# Convert date to datetime64
df1['date'] = pd.to_datetime(df1['date'])
df1.dtypes

date        datetime64[ns]
Open                object
High                object
Low                 object
Close               object
AdjClose            object
Volume              object
dtype: object

In [6]:
# Set the date an index
df1 = df1.set_index('date')
df1.head(3)

,Open,High,Low,Close,AdjClose,Volume
date,,,,,,
1998-01-02,74.5000,74.5000,72.6900,73.2500,11.9706,451500
1998-01-05,74.0000,74.8800,73.2500,74.8800,12.2369,770000
1998-01-06,74.9400,74.9400,71.7500,71.9400,11.7565,861600


In [7]:
# # Select the date range we want to plot (this one will be for the year 2000)
# df_2000 = df1.loc[(df1['date'] > '2000-07-01') & (df1['date'] < '2001-09-30')]
# df_2000.head(3)

In [8]:
# Add columns with year, month, and weekday name to the dataframe
df1['Year'] = df1.index.year
df['Month'] = df1.index.month
df1['Weekday Name'] = df1.index.weekday_name

# Display a random sampling of 5 rows
df1.sample(5, random_state=0)

,Open,High,Low,Close,AdjClose,Volume,Year,Weekday Name
date,,,,,,,,
2001-06-04,57.5000,57.5800,56.5000,57.4600,19.5883,992300,2001,Monday
1998-03-05,79.0000,79.4400,78.6300,79.1900,12.9928,914900,1998,Thursday
2000-09-11,54.0000,55.0600,53.5600,55.0000,18.5878,1758300,2000,Monday
2013-10-16,61.3300,62.3500,61.2300,62.2800,54.9931,2544400,2013,Wednesday
2015-09-11,61.3800,62.1100,61.3800,61.9200,57.0939,3464113,2015,Friday


In [9]:
# Select the date range we want to plot (this one will be for the year 2001)
df_2001 = df1.loc['2001-07-01':'2001-09-30', 'Close']
df_2001 = pd.DataFrame(df1)
df_2001.head(3)

,Open,High,Low,Close,AdjClose,Volume,Year,Weekday Name
date,,,,,,,,
1998-01-02,74.5000,74.5000,72.6900,73.2500,11.9706,451500,1998,Friday
1998-01-05,74.0000,74.8800,73.2500,74.8800,12.2369,770000,1998,Monday
1998-01-06,74.9400,74.9400,71.7500,71.9400,11.7565,861600,1998,Tuesday


In [17]:
#For the the column values to be used in the plots, we MUST convert them into numeric values first.
# the date was converted in datatime after creating the dataframe
df_2001.Open=pd.to_numeric(df_2001.Open)
df_2001.High=pd.to_numeric(df_2001.High)
df_2001.Low=pd.to_numeric(df_2001.Low)
df_2001.Close=pd.to_numeric(df_2001.Close)
df_2001.AdjClose=pd.to_numeric(df_2001.AdjClose)
df_2001.Volume=pd.to_numeric(df_2001.Volume)
df_2001.Year=pd.to_numeric(df_2001.Year)

In [16]:

# fig, axes = plt.subplots()
# ax.plot(df_2000_close['Close']
# df_2001['2000'].plot(ax=axes, label='2000', x=df_2001['2000'].index.date)
# df_2001['2001'].plot(ax=axes, label='2001', x=df_2001['2001'].index.date)
#         plt.show()

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook


fig, ax = plt.subplots(figsize=(12, 6))
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

ax.plot(df_2000_close['Close'],label='Close Prices Mean',  marker='.', linestyle='-', linewidth=1,)
ax.plot(df_2000_close['MA_5'], label='Moving Avg 5 days', linestyle='-', linewidth=1,)
ax.plot(df_2000_close['MA_15'], label='Moving Avg of 15 days', linestyle='-', linewidth=1,)

plt.title('Oil Stock Moving Average 07/2000 - 09/2000', fontsize=14, ha='right')
plt.ylabel('Year 2000 Crude Oil - Closing Prices')
plt.xlabel('Months 07/00 - 09/00')
plt.legend()
plt.margins(0.02)
plt.xticks(rotation=45)
ax.yaxis.set_major_formatter(plt.FuncFormatter('${:.0f}'.format))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
fig.autofmt_xdate()



In [18]:
# Select the column of data needed for plotting - Year 2000
df_2000_close = pd.DataFrame(df_2000[['date','Close']])
df_2000_close.Close.dtype
df_2000_close.Close = df_2000_close.Close.astype(float)
df_2000_close.head(3)

NameError: name 'df_2000' is not defined

In [15]:
# df_2000_close.groupby('date').plot(x='2000', y='Close')

In [ ]:
# Select the column of data needed for plotting - Year 2001
df_2001_close = pd.DataFrame(df_2001[['date','Close']])
df_2001_close.Close.dtype
df_2001_close.Close = df_2001_close.Close.astype(float)
df_2001_close.head(3)

In [ ]:
# Set the date an index
df_2000_close = df_2000_close.set_index('date')
df_2001_close = df_2001_close.set_index('date')

In [ ]:
# Use rolling method to plot moving averages
df_2000_close['MA_5'] = df_2000_close.Close.rolling(5).mean().shift()
df_2000_close['MA_15'] = df_2000_close.Close.rolling(15).mean().shift()
df_2000_close.head(3)

In [ ]:
# Use rolling method to plot moving averages
df_2001_close['MA_5'] = df_2001_close.Close.rolling(5).mean().shift()
df_2001_close['MA_15'] = df_2001_close.Close.rolling(15).mean().shift()
df_2001_close.head(3)

In [ ]:
#For the the column values to be used in the plots, we MUST convert them into numeric values first.
# the date was converted in datatime after creating the dataframe
df_2000_close.Close=pd.to_numeric(df_2000_close.Close)
df_2001_close.Close=pd.to_numeric(df_2001_close.Close)

In [ ]:
# Plot data and moving averages of 10 rolling standard deviation
# Moving Average plot for the year 2000

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.cbook as cbook


fig, ax = plt.subplots(figsize=(12, 6))
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

ax.plot(df_2000_close['Close'],label='Close Prices Mean',  marker='.', linestyle='-', linewidth=1,)
ax.plot(df_2000_close['MA_5'], label='Moving Avg 5 days', linestyle='-', linewidth=1,)
ax.plot(df_2000_close['MA_15'], label='Moving Avg of 15 days', linestyle='-', linewidth=1,)

plt.title('Oil Stock Moving Average 07/2000 - 09/2000', fontsize=14, ha='right')
plt.ylabel('Year 2000 Crude Oil - Closing Prices')
plt.xlabel('Months 07/00 - 09/00')
plt.legend()
plt.margins(0.02)
plt.xticks(rotation=45)
ax.yaxis.set_major_formatter(plt.FuncFormatter('${:.0f}'.format))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
fig.autofmt_xdate()


#---------------------------------------------------------------------------------------------------------------------
# Moving Average plot for the year 2001

fig, ax = plt.subplots(figsize=(12, 6))
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

ax.plot(df_2001_close['Close'],label='Close Prices Mean',  marker='.', linestyle='-', linewidth=1,)
ax.plot(df_2001_close['MA_5'], label='Moving Avg 5 days', linestyle='-', linewidth=1,)
ax.plot(df_2001_close['MA_15'], label='Moving Avg of 15 days', linestyle='-', linewidth=1,)

plt.title('Oil Stock Moving Average 07/2001 - 09/2001', fontsize=14, ha='right')
plt.ylabel('Year 2001 Crude Oil - Closing Prices')
plt.xlabel('Months 07/01 - 09/01')
plt.legend()
plt.margins(0.02)
plt.xticks(rotation=45)
ax.yaxis.set_major_formatter(plt.FuncFormatter('${:.0f}'.format))
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
fig.autofmt_xdate()
plt.show()

In [ ]:
# Compute log change (instantaneous rate of return) for the year 2000

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(14, 4))
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

df_2000_close["change"] = np.log(df_2000_close["Close"] / df_2000_close["Close"].shift())
plt.plot(df_2000_close.change)

plt.title('Rate of Return', fontsize=14, ha='center')
plt.ylabel('Rate of Return')
plt.xticks(rotation=45)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
fig.autofmt_xdate()


#-----------------------------------------------------------------------------------------------
# Rate or return plot for the year 2001

fig, ax = plt.subplots(figsize=(14, 4))
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

df_2001_close["change"] = np.log(df_2001_close["Close"] / df_2001_close["Close"].shift())
plt.plot(df_2001_close.change)

plt.title('Rate of Return', fontsize=14, ha='center')
plt.ylabel('Rate of Return')
plt.xticks(rotation=45)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
fig.autofmt_xdate()
plt.show()

In [ ]:
#  Compute rolling historical volatility, offset using .shift() method for the year 2000

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

fig, ax = plt.subplots(figsize=(12, 4))
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

df_2000_close['Volatility'] = df_2000_close.change.rolling(21).std().shift()
df_2000_close['Volatility'].plot()

plt.title('Rolling Historical Volatility', fontsize=14, ha='center')
plt.ylabel('Volatility')
plt.xticks(rotation=45)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
fig.autofmt_xdate()

#------------------------------------------------------------------------------------
# Compute rolling historical volatility, offset using .shift() method for the year 2001

fig, ax = plt.subplots(figsize=(12, 4))
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

df_2001_close['Volatility'] = df_2001_close.change.rolling(21).std().shift()
df_2001_close['Volatility'].plot()

plt.title('Rolling Historical Volatility', fontsize=14, ha='center')
plt.ylabel('Volatility')
plt.xticks(rotation=45)
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
fig.autofmt_xdate()
plt.show()


In [ ]:
# price analysis
#offset expected change 1-day
df_2000_close['exp_chng'] = df_2000_close['Volatility'] * df_2000_close['Close'].shift()
df_2000_close['actual_chng'] = df_2000_close['Close'] - df_2000_close['Close'].shift()
#running more than once will 
df_2000_close = pd.DataFrame(df_2000_close.iloc[22:])
df_2000_close['Magnitude'] = df_2000_close['actual_chng'] / df_2000_close['exp_chng'] 
df_2000_close['abs_magni'] = np.abs(df_2000_close['Magnitude'])
df_2000_close.head(2)

In [ ]:
# price analysis
#offset expected change 1-day
df_2001_close['exp_chng'] = df_2001_close['Volatility'] * df_2001_close['Close'].shift()
df_2001_close['actual_chng'] = df_2001_close['Close'] - df_2001_close['Close'].shift()
#running more than once will 
df_2001_close = pd.DataFrame(df_2001_close.iloc[22:])
df_2001_close['Magnitude'] = df_2001_close['actual_chng'] / df_2001_close['exp_chng'] 
df_2001_close['abs_magni'] = np.abs(df_2001_close['Magnitude'])
df_2001_close.head(2)

In [ ]:
fig, ax = plt.subplots()
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

plt.scatter(df_2000_close['actual_chng'], df_2000_close['abs_magni'])

plt.title('Price Analysis', fontsize=14, ha='center')
plt.ylabel('Magnitude')
plt.xlabel('Actual Change')

#---------------------------------------------------------------------------

fig, ax = plt.subplots()
plt.grid(True, 'major', 'y', ls='--', lw=.5, c='k', alpha=.3)

plt.scatter(df_2001_close['actual_chng'], df_2001_close['abs_magni'])

plt.title('Price Analysis', fontsize=14, ha='center')
plt.ylabel('Magnitude')
plt.xlabel('Actual Change')
plt.show()